# 🖼️ Trellis NIM - Image to 3D Generation

Transform your images into 3D models using NVIDIA Trellis NIM!

## Prerequisites:
- Run the `trellis_nim_simple.ipynb` notebook first to deploy the NIM container
- Make sure the Trellis NIM container is running

## Steps:
0. **Install Dependencies** - Install required Python packages
1. **Container Check** - Verify Trellis NIM is running
2. **Upload Image** - Select and upload your image
3. **Generate 3D Model** - Convert image to 3D GLB file

---


## Step 0: Install Required Dependencies

First, let's install the necessary Python packages for image upload and processing:


## 🔧 Fix for ModuleNotFoundError: ipywidgets

If you're getting the `ModuleNotFoundError: No module named 'ipywidgets'` error, here are the solutions:


In [1]:
import sys

print("📦 Installing required packages...")

# Install required packages
required_packages = [
    "ipywidgets",
    "Pillow", 
    "requests"
]

for package in required_packages:
    print(f"Installing {package}...")
    !{sys.executable} -m pip install {package} -q
    
print("✅ All packages installed successfully!")
print("💡 If running in JupyterLab, you may need to restart the kernel and enable widget extensions")
print("💡 Run: jupyter labextension install @jupyter-widgets/jupyterlab-manager")


📦 Installing required packages...
Installing ipywidgets...
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Installing Pillow...
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Installing requests...
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
✅ All packages installed successfully!
💡 If running in JupyterLab, you may need to restart the kernel and enable widget extensions
💡 Run: jupyter labextension install @jupyter-widgets/jupyterlab-manager


In [ ]:
# 🚨 CRITICAL FIX: Missing pip module in virtual environment

import sys
import subprocess
import os

print("🔍 Diagnosing Python environment...")
print(f"Python executable: {sys.executable}")
print(f"Virtual environment detected: {'.venv' in sys.executable}")

# Check if pip is available
def check_pip():
    try:
        result = subprocess.run([sys.executable, '-m', 'pip', '--version'], 
                              capture_output=True, text=True)
        if result.returncode == 0:
            print(f"✅ pip is available: {result.stdout.strip()}")
            return True
        else:
            print(f"❌ pip check failed: {result.stderr.strip()}")
            return False
    except Exception as e:
        print(f"❌ Cannot check pip: {e}")
        return False

if not check_pip():
    print("\n🔧 Installing pip in virtual environment...")
    
    # Method 1: Try to install pip using ensurepip
    try:
        result = subprocess.run([sys.executable, '-m', 'ensurepip', '--upgrade'], 
                              capture_output=True, text=True)
        if result.returncode == 0:
            print("✅ pip installed successfully using ensurepip")
        else:
            print(f"⚠️ ensurepip failed: {result.stderr}")
            
            # Method 2: Try direct pip installation
            print("🔄 Trying alternative pip installation...")
            import urllib.request
            
            # Download get-pip.py
            get_pip_url = 'https://bootstrap.pypa.io/get-pip.py'
            print(f"📥 Downloading get-pip.py from {get_pip_url}")
            
            with urllib.request.urlopen(get_pip_url) as response:
                get_pip_script = response.read()
            
            with open('get-pip.py', 'wb') as f:
                f.write(get_pip_script)
            
            # Run get-pip.py
            result = subprocess.run([sys.executable, 'get-pip.py'], 
                                  capture_output=True, text=True)
            
            if result.returncode == 0:
                print("✅ pip installed successfully using get-pip.py")
                os.remove('get-pip.py')  # Clean up
            else:
                print(f"❌ get-pip.py installation failed: {result.stderr}")
                
    except Exception as e:
        print(f"❌ Could not install pip: {e}")
        print("💡 Manual solution: Run this in terminal:")
        print(f"   {sys.executable} -m ensurepip --upgrade")
        print("   or")
        print(f"   curl https://bootstrap.pypa.io/get-pip.py -o get-pip.py")
        print(f"   {sys.executable} get-pip.py")

# Verify pip is now working
print("\n🔍 Final pip check...")
if check_pip():
    print("🎉 pip is now ready! You can proceed with package installation.")
else:
    print("❌ pip is still not available. Manual installation required.")


In [ ]:
# 🔧 Improved Package Installation (Run after fixing pip above)

import sys
import subprocess
import importlib

def install_package(package_name, import_name=None):
    """Install and verify a package"""
    if import_name is None:
        import_name = package_name
        
    # First check if already installed
    try:
        importlib.import_module(import_name)
        print(f"✅ {package_name} is already installed")
        return True
    except ImportError:
        pass
    
    print(f"📦 Installing {package_name}...")
    
    # Try installation
    try:
        result = subprocess.run([
            sys.executable, '-m', 'pip', 'install', 
            '--user', package_name
        ], capture_output=True, text=True, timeout=120)
        
        if result.returncode == 0:
            print(f"✅ Successfully installed {package_name}")
            
            # Verify installation
            try:
                importlib.import_module(import_name)
                print(f"✅ {package_name} import verified")
                return True
            except ImportError:
                print(f"⚠️ {package_name} installed but import failed - may need kernel restart")
                return True
                
        else:
            print(f"❌ Installation failed: {result.stderr}")
            return False
            
    except Exception as e:
        print(f"❌ Installation error: {e}")
        return False

print("🚀 Installing required packages for Trellis image-to-3D...")
print("=" * 60)

# Package mapping: install_name -> import_name
packages = [
    ("ipywidgets", "ipywidgets"),
    ("Pillow", "PIL"),
    ("requests", "requests")
]

success_count = 0
for install_name, import_name in packages:
    if install_package(install_name, import_name):
        success_count += 1

print("=" * 60)
print(f"📊 Installation Results: {success_count}/{len(packages)} packages successful")

if success_count == len(packages):
    print("🎉 All packages installed successfully!")
    print("🔄 If you get import errors in Step 1, restart the kernel and try again")
else:
    print(f"⚠️ {len(packages) - success_count} packages failed to install")
    print("💡 Try running this cell again or install manually")
    
print("\n🎯 Next steps:")
print("1. If all packages installed: proceed to Step 1")
print("2. If import errors occur: restart kernel, then run Step 1")
print("3. If installations failed: check the error messages above")


## Step 1: Check NIM Container Connectivity

Now let's verify that the Trellis NIM container is running and accessible:


In [2]:
import requests
import json
import base64
import subprocess
from IPython.display import display, HTML, Image as IPImage
import ipywidgets as widgets
from ipywidgets import FileUpload, Button, Output, VBox, HBox
import io
from PIL import Image
import os

print("🔍 Checking NIM container status...")

# Check if container is running
container_check = !docker ps | grep nim-server
if not container_check:
    print("❌ Trellis NIM container is not running!")
    print("💡 Please run the 'trellis_nim_simple.ipynb' notebook first to deploy the container")
    print("💡 Or start it manually with the docker run command from that notebook")
else:
    print("✅ NIM container is running")
    print(f"Container info: {container_check[0]}")

# Test connectivity
print("\n🌐 Testing NIM container connectivity...")
try:
    response = requests.get("http://localhost:8000/health", timeout=5)
    if response.status_code == 200:
        print("✅ NIM container is responding to health checks")
    else:
        print(f"⚠️ NIM container responded with status code: {response.status_code}")
except requests.RequestException as e:
    print(f"❌ Cannot connect to NIM container: {e}")
    print("💡 Make sure the container is fully initialized (may take 2-5 minutes after start)")

print("\n📊 Recent container logs:")
!docker logs nim-server --tail 5


🔍 Checking NIM container status...
✅ NIM container is running
Container info: Cannot connect to the Docker daemon at unix:///Users/isoltysik/.docker/run/docker.sock. Is the docker daemon running?

🌐 Testing NIM container connectivity...
❌ Cannot connect to NIM container: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /health (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x10dd81ed0>: Failed to establish a new connection: [Errno 61] Connection refused'))
💡 Make sure the container is fully initialized (may take 2-5 minutes after start)

📊 Recent container logs:
Cannot connect to the Docker daemon at unix:///Users/isoltysik/.docker/run/docker.sock. Is the docker daemon running?


## 🔧 Widget Rendering Fix

If widgets show as text (like `VBox(children=...)`) instead of interactive controls, this cell will fix it:


In [ ]:
# 🔍 Diagnose and Fix Widget Rendering Issues

import sys
import subprocess
import IPython
from IPython.display import display, HTML
import ipywidgets as widgets

print("🔍 Diagnosing Jupyter widget rendering...")
print("=" * 50)

print(f"📍 IPython version: {IPython.__version__}")
print(f"📍 ipywidgets version: {widgets.__version__}")

# Test if widgets can render
print("\n🧪 Testing widget rendering...")

# Create a simple test widget
test_button = widgets.Button(description="Test Widget")
test_output = widgets.Output()

print("Creating test button widget...")
display(test_button)

# Check if we can see the comm manager
try:
    from ipykernel.comm import Comm
    print("✅ Comm manager available")
except ImportError:
    print("❌ Comm manager not available")

# Check widget extension status in different environments
print("\n🔧 Widget Extension Checks:")

# Check for JupyterLab extensions
try:
    result = subprocess.run(['jupyter', 'labextension', 'list'], 
                          capture_output=True, text=True, timeout=10)
    if '@jupyter-widgets/jupyterlab-manager' in result.stdout:
        print("✅ JupyterLab widget extension is installed")
    else:
        print("❌ JupyterLab widget extension NOT installed")
        print("💡 Install it: jupyter labextension install @jupyter-widgets/jupyterlab-manager")
except:
    print("⚠️ Cannot check JupyterLab extensions (command not available)")

# Check notebook extensions  
try:
    result = subprocess.run(['jupyter', 'nbextension', 'list'], 
                          capture_output=True, text=True, timeout=10)
    if 'jupyter-js-widgets/extension' in result.stdout and 'enabled' in result.stdout:
        print("✅ Jupyter Notebook widget extension is enabled")
    else:
        print("❌ Jupyter Notebook widget extension NOT enabled")
        print("💡 Enable it: jupyter nbextension enable --py widgetsnbextension")
except:
    print("⚠️ Cannot check notebook extensions (command not available)")

print("\n🔄 Solutions if widgets aren't rendering:")
print("1. RESTART KERNEL: Kernel → Restart (most common fix)")
print("2. JupyterLab: jupyter labextension install @jupyter-widgets/jupyterlab-manager")
print("3. Jupyter Notebook: jupyter nbextension enable --py widgetsnbextension")
print("4. If still not working: use the alternative upload method below")

# Alternative test - inline display
print("\n📋 Alternative: If widgets don't work, we'll use manual file upload below")


## 🔄 Alternative Image Upload (No Widgets Required)

Use this method if the widgets in Step 2 don't work properly:


In [ ]:
# 🎯 Manual Image Upload Alternative (Widget-Free)

import os
import base64
from PIL import Image
import io
from IPython.display import display, Image as IPImage

print("📋 Manual Image Upload Instructions:")
print("=" * 50)
print("1. Copy your image file to this notebook's directory")
print("2. Update the IMAGE_PATH below with your image filename")
print("3. Run this cell to load the image")
print("")

# ⚠️ CHANGE THIS PATH TO YOUR IMAGE FILE ⚠️
IMAGE_PATH = "your_image.jpg"  # Change this to your image filename

print(f"🎯 Current working directory: {os.getcwd()}")
print("📁 Available image files in this directory:")

# List common image files in the current directory
image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.webp']
image_files = [f for f in os.listdir('.') if any(f.lower().endswith(ext) for ext in image_extensions)]

if image_files:
    print("   Available images:")
    for i, img_file in enumerate(image_files, 1):
        print(f"   {i}. {img_file}")
    print(f"\n💡 To use an image, change IMAGE_PATH above to: '{image_files[0]}'")
else:
    print("   ❌ No image files found in current directory")
    print("   💡 Copy an image file here first")

# Global variables for compatibility with Step 3
uploaded_image_data = None
uploaded_image_name = None

# Try to load the specified image
if os.path.exists(IMAGE_PATH) and IMAGE_PATH != "your_image.jpg":
    try:
        print(f"\n📤 Loading image: {IMAGE_PATH}")
        
        # Read the image file
        with open(IMAGE_PATH, 'rb') as f:
            uploaded_image_data = f.read()
        
        uploaded_image_name = IMAGE_PATH
        
        # Display image info
        print(f"✅ Successfully loaded: {uploaded_image_name}")
        print(f"📊 File size: {len(uploaded_image_data) / 1024:.2f} KB")
        
        # Display the image
        image = Image.open(io.BytesIO(uploaded_image_data))
        print(f"📏 Dimensions: {image.width} x {image.height}")
        print(f"🎨 Format: {image.format}")
        
        # Show a thumbnail
        display_image = image.copy()
        if display_image.width > 400 or display_image.height > 400:
            display_image.thumbnail((400, 400), Image.Resampling.LANCZOS)
        
        img_buffer = io.BytesIO()
        display_image.save(img_buffer, format='PNG')
        img_buffer.seek(0)
        
        display(IPImage(data=img_buffer.getvalue()))
        print("✅ Image ready for 3D generation!")
        
    except Exception as e:
        print(f"❌ Error loading image: {e}")
        uploaded_image_data = None
        uploaded_image_name = None
        
elif IMAGE_PATH == "your_image.jpg":
    print(f"\n⚠️ Please update IMAGE_PATH variable above with your actual image filename")
else:
    print(f"\n❌ Image file not found: {IMAGE_PATH}")
    print(f"💡 Make sure the file exists in: {os.getcwd()}")

print(f"\n🎯 Status: {'✅ Image loaded and ready' if uploaded_image_data else '❌ No image loaded'}")


## Step 2: Image Upload Interface

Upload an image that you want to convert to a 3D model:


In [ ]:
# Create image upload widget
upload_widget = FileUpload(
    accept='.jpg,.jpeg,.png,.bmp,.tiff,.webp',  # Accept common image formats
    multiple=False,  # Single file upload
    description='Choose Image'
)

# Output widget to display uploaded image
image_display = Output()

# Global variable to store uploaded image data
uploaded_image_data = None
uploaded_image_name = None

def on_upload_change(change):
    global uploaded_image_data, uploaded_image_name
    
    with image_display:
        image_display.clear_output()
        
        if upload_widget.value:
            # Get the uploaded file
            uploaded_file = list(upload_widget.value.values())[0]
            uploaded_image_name = list(upload_widget.value.keys())[0]
            uploaded_image_data = uploaded_file['content']
            
            print(f"📁 Uploaded: {uploaded_image_name}")
            print(f"📊 File size: {len(uploaded_image_data) / 1024:.2f} KB")
            
            # Display the uploaded image
            try:
                image = Image.open(io.BytesIO(uploaded_image_data))
                
                # Resize image for display if it's too large
                display_image = image.copy()
                if display_image.width > 400 or display_image.height > 400:
                    display_image.thumbnail((400, 400), Image.Resampling.LANCZOS)
                
                # Convert to bytes for display
                img_buffer = io.BytesIO()
                display_image.save(img_buffer, format='PNG')
                img_buffer.seek(0)
                
                display(IPImage(data=img_buffer.getvalue()))
                print(f"📏 Original dimensions: {image.width} x {image.height}")
                print(f"🎨 Image format: {image.format}")
                print("✅ Image ready for 3D generation!")
                
            except Exception as e:
                print(f"❌ Error processing image: {e}")
        else:
            uploaded_image_data = None
            uploaded_image_name = None
            print("No image selected")

# Attach the event handler
upload_widget.observe(on_upload_change, names='value')

# Display the upload interface
print("📤 Select an image to upload:")
print("Supported formats: JPG, PNG, BMP, TIFF, WebP")
print("")
display(VBox([upload_widget, image_display]))


## ⚠️ If Step 2 Widgets Show as Text

If you see `VBox(children=...)` instead of an upload button, here's the solution:


In [ ]:
# 🚨 WIDGET RENDERING ISSUE - Quick Fix

print("🔧 Widget Rendering Problem Detected!")
print("=" * 50)
print("If you see 'VBox(children=...)' text instead of an upload button:")
print("")
print("✅ SOLUTION 1 (Most Effective):")
print("   1. Restart your kernel: Kernel → Restart")
print("   2. Re-run all cells from the beginning")
print("   3. Widgets should render properly after restart")
print("")
print("✅ SOLUTION 2 (Alternative):")
print("   Use the 'Alternative Image Upload' method above")
print("   - Copy your image to this directory")
print("   - Edit the IMAGE_PATH variable")
print("   - Run that cell instead")
print("")
print("✅ SOLUTION 3 (JupyterLab Extension):")
print("   Run these commands in terminal if using JupyterLab:")
print("   jupyter labextension install @jupyter-widgets/jupyterlab-manager")
print("   jupyter labextension enable @jupyter-widgets/jupyterlab-manager")
print("")
print("🎯 RECOMMENDED: Try Solution 1 (restart kernel) first!")
print("   Then use Solution 2 (manual upload) as backup")

# Test widget rendering one more time
try:
    import ipywidgets as widgets
    from IPython.display import display
    
    test_widget = widgets.HTML(value="<b>🧪 Test Widget - If you see this styled text, widgets work!</b>")
    print("\n🧪 Testing widget rendering:")
    display(test_widget)
    print("✅ If you see styled text above, widgets are working!")
    
except Exception as e:
    print(f"\n❌ Widget test failed: {e}")
    print("💡 Definitely use the Alternative Upload method instead")


## Step 3: Generate 3D Model from Image

Convert your uploaded image into a 3D model:


In [ ]:
# Configuration for 3D generation
OUTPUT_FILE = "image_to_3d_result.glb"  # Change this to customize output filename
SEED = 0  # Change this for different variations (0 = random)

# Check if image is uploaded
if uploaded_image_data is None:
    print("❌ Please upload an image first using the widget above!")
else:
    print(f"🖼️ Processing image: {uploaded_image_name}")
    print(f"💾 Output file: {OUTPUT_FILE}")
    print(f"🎲 Seed: {SEED}")
    
    try:
        # Convert image to base64 for API request
        image_base64 = base64.b64encode(uploaded_image_data).decode('utf-8')
        
        # Create the JSON payload for image-to-3D
        json_payload = json.dumps({
            "image": image_base64,
            "seed": SEED
        })
        
        print("\n📡 Sending image-to-3D generation request to Trellis NIM...")
        print("⏳ This may take 2-5 minutes...")
        
        # Execute curl command for image-to-3D inference
        curl_cmd = [
            'curl', '-X', 'POST', 'http://localhost:8000/v1/infer',
            '-H', 'Accept: application/json',
            '-H', 'Content-Type: application/json',
            '-d', json_payload,
            '--silent'
        ]
        
        # Run the curl command
        result = subprocess.run(curl_cmd, capture_output=True, text=True, timeout=600)  # 10 minute timeout
        
        if result.returncode != 0:
            print(f"❌ Curl command failed with return code: {result.returncode}")
            print(f"Error: {result.stderr}")
            print(f"Command: {' '.join(curl_cmd)}")
        else:
            response_text = result.stdout.strip()
            print(f"📨 Raw response length: {len(response_text)} characters")
            
            if not response_text:
                print("❌ Empty response from server")
            elif response_text.startswith('<!DOCTYPE') or '<html' in response_text:
                print("❌ Received HTML response instead of JSON - check if container supports image-to-3D")
                print("💡 Some NIM versions may only support text-to-3D")
            else:
                try:
                    # Parse JSON response
                    response_data = json.loads(response_text)
                    print("✅ Valid JSON response received")
                    
                    # Check for artifacts in response
                    if 'artifacts' in response_data and len(response_data['artifacts']) > 0:
                        artifact = response_data['artifacts'][0]
                        if 'base64' in artifact:
                            base64_data = artifact['base64']
                            
                            # Decode and save the GLB file
                            glb_data = base64.b64decode(base64_data)
                            
                            with open(OUTPUT_FILE, 'wb') as f:
                                f.write(glb_data)
                            
                            print(f"✅ SUCCESS! 3D model generated from your image!")
                            print(f"💾 3D model saved as: {OUTPUT_FILE}")
                            print(f"📊 File size: {len(glb_data) / 1024:.2f} KB")
                            print(f"🎯 You can now open {OUTPUT_FILE} in any 3D viewer or Blender!")
                            
                            # Show file info
                            print(f"\n📁 Generated files:")
                            !ls -la *.glb
                            
                        else:
                            print("❌ No base64 data in artifact")
                            print(f"Artifact keys: {list(artifact.keys())}")
                    else:
                        print("❌ No artifacts found in response")
                        print(f"Response keys: {list(response_data.keys())}")
                        if 'error' in response_data:
                            print(f"Error from server: {response_data['error']}")
                        
                        # Show full response for debugging
                        print(f"\n🔍 Full response for debugging:")
                        print(json.dumps(response_data, indent=2)[:1000])
                            
                except json.JSONDecodeError as e:
                    print(f"❌ Failed to parse JSON response: {e}")
                    print(f"Raw response preview: {response_text[:500]}...")

    except subprocess.TimeoutExpired:
        print("❌ Request timed out (>10 minutes)")
        print("💡 Image-to-3D generation may take longer than text-to-3D")
    except Exception as e:
        print(f"❌ Unexpected error: {e}")
        import traceback
        traceback.print_exc()


## 🎉 Success!

### To generate more 3D models from images:
1. Upload a new image using the file upload widget above
2. Optionally change the `OUTPUT_FILE` and `SEED` in the generation cell
3. Run the generation cell again

### Tips for best results:
- Use images with clear, well-lit subjects
- Objects with distinct shapes work better than complex scenes
- Single objects tend to generate better 3D models than cluttered images
- Try different seeds for variations of the same image

### Image format support:
- **Recommended**: PNG, JPG/JPEG
- **Also supported**: BMP, TIFF, WebP
- **Resolution**: Higher resolution images may produce better results

---


## Utility Commands

Helpful commands for managing your 3D generation workflow:


In [ ]:
# List all generated GLB files
print("📁 Generated 3D model files:")
!ls -la *.glb 2>/dev/null || echo "No GLB files found"


In [ ]:
# Check NIM container status and logs
print("🐳 Container status:")
!docker ps | grep nim-server
print("\n📊 Recent container logs:")
!docker logs nim-server --tail 10


In [ ]:
# Test image upload functionality
print("🧪 Testing upload state:")
if uploaded_image_data is not None:
    print(f"✅ Image loaded: {uploaded_image_name}")
    print(f"📊 Size: {len(uploaded_image_data) / 1024:.2f} KB")
    
    # Show image info
    try:
        image = Image.open(io.BytesIO(uploaded_image_data))
        print(f"📏 Dimensions: {image.width} x {image.height}")
        print(f"🎨 Format: {image.format}")
        print(f"🌈 Mode: {image.mode}")
    except Exception as e:
        print(f"❌ Error reading image: {e}")
else:
    print("❌ No image currently loaded")
    print("💡 Upload an image using the widget above")


In [ ]:
# Create output directory for organized file management
output_dir = "output"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"📁 Created output directory: {output_dir}/")
else:
    print(f"📁 Output directory exists: {output_dir}/")

# Move GLB files to output directory
glb_files = !ls *.glb 2>/dev/null || echo ""
if glb_files and glb_files[0]:  # Check if any GLB files exist
    print("\n📦 Moving GLB files to output directory...")
    !mv *.glb output/ 2>/dev/null
    print("✅ GLB files moved to output/")
    !ls -la output/*.glb
else:
    print("\n📭 No GLB files to move")
